In [ ]:
# Import the libraries- Data manipulation

import numpy as np
import pandas as pd
import datetime

# To calculate Greeks
import mibian

# For Plotting
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

#module 2-determine top 5 constituents

HDFCBANK_Lot_Size = 500

ICICIBANK_Wt = 0.173
ICICIBANK_Lot_Size = 2500

KOTAKBANK_Wt = 0.123
KOTAKBANK_Lot_Size = 800

SBIN_Wt = 0.102
SBIN_Lot_Size = 3000

AXISBANK_Wt = 0.08
AXISBANK_Lot_Size = 1200

BankNifty_Wt = 1.0
BankNifty_Lot_Size = 40

#module 3- Read data

def read_data(inst_name):
    opt = pd.read_csv(inst_name + ".csv")
    fut = pd.read_csv('../data_modules/BankNifty_Options_Data.csv')
    opt = opt[opt['Open Int'] != 0]
    opt = opt[['Symbol', 'Date', 'Expiry', 'Option Type',
               'Strike Price', 'Close', 'futures_price']]
    return opt


BankNifty_Opt = read_data('../data_modules/BankNifty_Options_Data')
BankNifty_Opt = BankNifty_Opt[(BankNifty_Opt.Symbol == 'BANKNIFTY')]
BankNifty_Opt.head()

#module 4- calculate time to expiry

def time_to_expiry(opt):
    opt.Expiry = pd.to_datetime(opt.Expiry)
    opt.Date = pd.to_datetime(opt.Date)
    opt['time_to_expiry'] = (opt.Expiry - opt.Date).dt.days
    return opt


BankNifty_Opt = time_to_expiry(BankNifty_Opt)
BankNifty_Opt.head()

#module 5-Determine ATM strike price
def atm_strike_price(opt):
    opt['strike_distance'] = np.abs(opt.futures_price - opt['Strike Price'])
    df = opt.groupby(['Date'])['strike_distance'].min().to_frame()
    df.index.column = 0
    opt = pd.merge(opt, df)
    opt = opt[(np.abs(opt.futures_price - opt['Strike Price'])
               == opt.strike_distance)]
    opt = opt.drop('strike_distance', 1)
    opt = opt.drop_duplicates(subset=['Date', 'Expiry', 'Option Type'])
    return opt


full_BankNifty_opt = BankNifty_Opt
BankNifty_Opt = atm_strike_price(BankNifty_Opt)
BankNifty_Opt.tail()

#module 6- calculate pnl from long straddle
def daily_pnl(opt, full_opt):
    opt['next_day_close'] = np.nan
    opt.sort_values('Date', inplace=True, ascending=True)
    for i in range(0, len(opt)-2):
        strike_price = opt.iloc[i]['Strike Price']
        trade_date = opt.iloc[i]['Date']
        next_trading_date = opt[(opt.Date > trade_date)
                                & (opt.Date <= trade_date + datetime.timedelta(days=20)
                                   )].iloc[0]['Date']
        option_type = opt.iloc[i]['Option Type']

        if opt.iloc[i]['time_to_expiry'] != 0:
            opt.iloc[i, opt.columns.get_loc('next_day_close')] = full_opt[(full_opt['Strike Price'] == strike_price) &
                                                                          (full_opt['Date'] == next_trading_date) &
                                                                          (full_opt['Option Type']
                                                                           == option_type)
                                                                          ].iloc[0]['Close']
        else:
            # This is done because on expiry day the next day price doesn't exists
            opt.iloc[i, opt.columns.get_loc(
                'next_day_close')] = opt.iloc[i]['Close']

    opt['daily_straddle_pnl'] = opt.next_day_close - opt.Close
    return opt


BankNifty_Opt = daily_pnl(BankNifty_Opt, full_BankNifty_opt)
BankNifty_Opt.head()

#module 7- compute implied volatility
def implied_volatility_options(opt):
    opt['IV'] = np.nan
    opt = opt.iloc[:3]
    opt.loc[(opt.time_to_expiry == 0), 'time_to_expiry'] = 0.0000001
    for i in range(0, len(opt)):
        if opt.iloc[i]['Option Type'] == 'CE':
            opt.iloc[i, opt.columns.get_loc('IV')] = mibian.BS([opt.iloc[i]['futures_price'],
                                                                opt.iloc[i]['Strike Price'],
                                                                0,
                                                                opt.iloc[i]['time_to_expiry']],
                                                               callPrice=opt.iloc[i]['Close']
                                                               ).impliedVolatility
        else:
            opt.iloc[i, opt.columns.get_loc('IV')] = mibian.BS([opt.iloc[i]['futures_price'],
                                                                opt.iloc[i]['Strike Price'],
                                                                0,
                                                                opt.iloc[i]['time_to_expiry']],
                                                               putPrice=opt.iloc[i]['Close']
                                                               ).impliedVolatility
    return opt


# Since this process is computationally heavy, we will calculate the implied volatility for first 3 rows and 
#import the remaining implied volatility data from the BankNifty_Preprocessed_Options_Data csv file.
#In BankNifty_Preprocessed_Options_Data csv file, we have already computed and stored the implied volatility data in it.

BankNifty_Opt.iloc[:3] = implied_volatility_options(BankNifty_Opt)
opt_IV = pd.read_csv(
    '../data_modules/BankNifty_Preprocessed_Options_Data.csv', index_col=0)
BankNifty_Opt = opt_IV[(opt_IV.Symbol == BankNifty_Opt.Symbol.iloc[0])]
BankNifty_Opt.tail()

#module 8-Compute delta

def delta_options(opt):
    opt['delta'] = np.nan
    opt = opt.iloc[:3]
    for i in range(0, len(opt)):
        if opt.iloc[i]['Option Type'] == 'CE':
            opt.iloc[i, opt.columns.get_loc('delta')] = mibian.BS([opt.iloc[i]['futures_price'],
                                                                   opt.iloc[i]['Strike Price'],
                                                                   0,
                                                                   opt.iloc[i]['time_to_expiry']],
                                                                  volatility=opt.iloc[i]['IV']
                                                                  ).callDelta
        else:
            opt.iloc[i, opt.columns.get_loc('delta')] = mibian.BS([opt.iloc[i]['futures_price'],
                                                                   opt.iloc[i]['Strike Price'],
                                                                   0,
                                                                   opt.iloc[i]['time_to_expiry']],
                                                                  volatility=opt.iloc[i]['IV']
                                                                  ).putDelta
    return opt



#Since this process is computationally heavy, we will calculate the Delta for first 3 rows and 
#the remaining Delta data is already imported from the BankNifty_Preprocessed_Options_Data csv file in the previous step.

BankNifty_Opt.iloc[:3] = delta_options(BankNifty_Opt)
BankNifty_Opt.tail(6)

#module 9- Read data and preprocess banknifty constituents stocks data
def read_constituents_data(inst_name):
    opt = read_data(inst_name)
    opt = time_to_expiry(opt)

    HDFCBANK_Opt = opt[(opt.Symbol == 'HDFCBANK')]
    ICICIBANK_Opt = opt[(opt.Symbol == 'ICICIBANK')]
    KOTAKBANK_Opt = opt[(opt.Symbol == 'KOTAKBANK')]
    SBIN_Opt = opt[(opt.Symbol == 'SBIN')]
    AXISBANK_Opt = opt[(opt.Symbol == 'AXISBANK')]

    return HDFCBANK_Opt, ICICIBANK_Opt, KOTAKBANK_Opt, SBIN_Opt, AXISBANK_Opt


HDFCBANK_Opt, ICICIBANK_Opt, KOTAKBANK_Opt, SBIN_Opt, AXISBANK_Opt = read_constituents_data(
    '../data_modules/BankNifty_Options_Data')
HDFCBANK_Opt.head()

def preprocess(opt):
    full_opt = opt
    opt = atm_strike_price(opt)
    opt = daily_pnl(opt, full_opt)
    return opt


HDFCBANK_Opt = preprocess(HDFCBANK_Opt)
ICICIBANK_Opt = preprocess(ICICIBANK_Opt)
KOTAKBANK_Opt = preprocess(KOTAKBANK_Opt)
SBIN_Opt = preprocess(SBIN_Opt)
AXISBANK_Opt = preprocess(AXISBANK_Opt)
HDFCBANK_Opt.head()

def greeks(opt):
    
    #Since the process to compute the implied volatility and Delta is time consuming, 
    #we will import the implied volatility and Delta data from the BankNifty_Preprocessed_Options_Data csv file.
    
    #opt = implied_volatility_options(opt)
    #opt = delta_options(opt)
    opt_IV = pd.read_csv(
        '../data_modules/BankNifty_Preprocessed_Options_Data.csv', index_col=0)
    opt = opt_IV[(opt_IV.Symbol == opt.Symbol.iloc[0])]
    return opt


HDFCBANK_Opt = greeks(HDFCBANK_Opt)
ICICIBANK_Opt = greeks(ICICIBANK_Opt)
KOTAKBANK_Opt = greeks(KOTAKBANK_Opt)
SBIN_Opt = greeks(SBIN_Opt)
AXISBANK_Opt = greeks(AXISBANK_Opt)
HDFCBANK_Opt.tail()




#module 10-Delta Hedging
HDFCBANK_delta = HDFCBANK_Opt.groupby(['Date'])['delta'].sum().to_frame()
HDFCBANK_delta.tail()

#module 11- Implied dirty corelation
def implied_dirty_correlation():
    BankNifty_IV = BankNifty_Opt.groupby(['Date'])['IV'].mean().to_frame()

    HDFCBANK_IV = HDFCBANK_Opt.groupby(['Date'])['IV'].mean().to_frame()
    ICICIBANK_IV = ICICIBANK_Opt.groupby(['Date'])['IV'].mean().to_frame()
    KOTAKBANK_IV = KOTAKBANK_Opt.groupby(['Date'])['IV'].mean().to_frame()
    SBIN_IV = SBIN_Opt.groupby(['Date'])['IV'].mean().to_frame()
    AXISBANK_IV = AXISBANK_Opt.groupby(['Date'])['IV'].mean().to_frame()

    weighted_average_constituents_vol = HDFCBANK_IV * HDFCBANK_Wt + \
        SBIN_IV * SBIN_Wt + \
        ICICIBANK_IV * ICICIBANK_Wt + \
        KOTAKBANK_IV * KOTAKBANK_Wt + \
        AXISBANK_IV * AXISBANK_Wt

    return (BankNifty_IV/weighted_average_constituents_vol)**2


df = implied_dirty_correlation()
df = df.rename(columns={'IV': 'implied_correlation'})
df.plot()
plt.show()

#module 12- backtest
def trading_signal(df):
    df.index.column = 0

    lookback = 5
    # Moving Average
    df['moving_average'] = df['implied_correlation'].rolling(lookback).mean()
    # Moving Standard Deviation
    df['moving_std_dev'] = df['implied_correlation'].rolling(lookback).std()

    df['upper_band'] = df.moving_average + 0.5*df.moving_std_dev
    df['lower_band'] = df.moving_average - 0.5*df.moving_std_dev

    df['long_entry'] = df.implied_correlation < df.lower_band
    df['long_exit'] = df.implied_correlation >= df.moving_average

    df['short_entry'] = df.implied_correlation > df.upper_band
    df['short_exit'] = df.implied_correlation <= df.moving_average

    df['positions_long'] = np.nan
    df.loc[df.long_entry, 'positions_long'] = 1
    df.loc[df.long_exit, 'positions_long'] = 0

    expiry_dates = HDFCBANK_Opt.Expiry.unique()
    df.loc[df.index.isin(expiry_dates), 'positions_short'] = 0

    df['positions_short'] = np.nan
    df.loc[df.short_entry, 'positions_short'] = -1
    df.loc[df.short_exit, 'positions_short'] = 0

    df.loc[df.index.isin(expiry_dates), 'positions_short'] = 0

    df = df.fillna(method='ffill')

    df['positions'] = df.positions_long + df.positions_short

    return df


df = trading_signal(df)
df.positions.tail()


#module 13- strategy pnl
def strategy_pnl(opt, df):
    opt = pd.merge(opt, df[['positions']], left_on='Date',
                   right_index=True, how='left')
    opt['strategy_pnl'] = opt.positions * opt.daily_straddle_pnl
    return opt


BankNifty_Opt = strategy_pnl(BankNifty_Opt, df)
BankNifty_Opt.head()

df.positions *= -1
HDFCBANK_Opt = strategy_pnl(HDFCBANK_Opt, df)
ICICIBANK_Opt = strategy_pnl(ICICIBANK_Opt, df)
KOTAKBANK_Opt = strategy_pnl(KOTAKBANK_Opt, df)
SBIN_Opt = strategy_pnl(SBIN_Opt, df)
AXISBANK_Opt = strategy_pnl(AXISBANK_Opt, df)
HDFCBANK_Opt.tail()

BankNifty_Ret = BankNifty_Opt.groupby(
    ['Date'])['strategy_pnl'].sum().to_frame()

HDFCBANK_Ret = HDFCBANK_Opt.groupby(['Date'])['strategy_pnl'].sum().to_frame()

ICICIBANK_Ret = ICICIBANK_Opt.groupby(
    ['Date'])['strategy_pnl'].sum().to_frame()

KOTAKBANK_Ret = KOTAKBANK_Opt.groupby(
    ['Date'])['strategy_pnl'].sum().to_frame()

SBIN_Ret = SBIN_Opt.groupby(['Date'])['strategy_pnl'].sum().to_frame()

AXISBANK_Ret = AXISBANK_Opt.groupby(['Date'])['strategy_pnl'].sum().to_frame()

BankNifty_Ret.tail()

HDFCBANK_Ret.tail()

def strategy_pnl():
    strategy_pnl = HDFCBANK_Ret.strategy_pnl * HDFCBANK_Lot_Size * HDFCBANK_Wt + \
        SBIN_Ret.strategy_pnl * SBIN_Lot_Size * SBIN_Wt + \
        AXISBANK_Ret.strategy_pnl * AXISBANK_Lot_Size * AXISBANK_Wt + \
        KOTAKBANK_Ret.strategy_pnl * KOTAKBANK_Lot_Size * KOTAKBANK_Wt + \
        ICICIBANK_Ret.strategy_pnl * ICICIBANK_Lot_Size * ICICIBANK_Wt + \
        BankNifty_Ret.strategy_pnl * BankNifty_Lot_Size * BankNifty_Wt
    return strategy_pnl.cumsum().shift(1)

strategy_pnl().plot(figsize=(10, 5))
plt.ylabel("Strategy PnL")
plt.show()














ModuleNotFoundError: No module named 'numpy'